# Volume Reversal Strategy

In this notebook, we will implement a trading strategy based on price movements corresponding to the trading volume of the security.

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import BATrader as ba

## Stock data

In [14]:
data = ba.tc.get_bar_ex_eod_db('700', dayback= 500)
data.head()

,Open,High,Low,Close,Vol
Date,,,,,
2018-07-31,358.905334,359.700692,351.349432,353.138988,3.719448e+07
2018-08-01,358.706494,359.104173,350.951753,352.940148,2.585878e+07
2018-08-02,350.752913,352.343629,338.424863,342.998172,4.101500e+07
2018-08-03,346.179604,352.542469,344.588888,347.770320,3.044742e+07
2018-08-06,355.723901,355.723901,347.969160,350.951753,2.157157e+07


## Price change for the last 5 days
We will calculate the difference between the current price of the stock and the price of the stock five days ago. We will use shift() function for fetching the data for days in the past. data['Close'].shift(1) will give us yesterday's closing price, data['Close'].shift(6) will give us the closing price 6 days earlier. The difference of the two will give us the price change for the last 5 days and store this in a new column called 'pr_change'.

In [15]:
data['pr_chg'] = data['Close'].shift(1) - data['Close'].shift(6)

## Standard deviation of price change
Next, we will calculate 100 days standard deviation of price change and store the value in a new column called 'std_100day' by using the rolling.std() function. This will help us to measure volatility of change in price for the stock.

In [16]:
data['std_100day'] = data['pr_chg'].rolling(window=100).std()

## Average volume traded
Now we will calculate the last 5 day's (previous week's) average traded volume by using the shift() and rolling.mean() functions and store the values in a new column '5d_avg_vol'. 

In [17]:
data['5d_avg_vol'] = data['Vol'].shift(1).rolling(window=5).mean()

## Average volume traded between last 5 to 10 days
Next, we will calculate the average traded volume for last 5 to 10 days by shifting the values in column '5d_avg_vol' and store this value in a new column 'past 5d_avg_vol'.

In [19]:
data['past 5d_avg_vol'] = data['5d_avg_vol'].shift(5)
data

,Open,High,Low,Close,Vol,pr_chg,std_100day,5d_avg_vol,past 5d_avg_vol
Date,,,,,,,,,
2018-07-31,358.905334,359.700692,351.349432,353.138988,3.719448e+07,NaN,NaN,NaN,NaN
2018-08-01,358.706494,359.104173,350.951753,352.940148,2.585878e+07,NaN,NaN,NaN,NaN
2018-08-02,350.752913,352.343629,338.424863,342.998172,4.101500e+07,NaN,NaN,NaN,NaN
2018-08-03,346.179604,352.542469,344.588888,347.770320,3.044742e+07,NaN,NaN,NaN,NaN
2018-08-06,355.723901,355.723901,347.969160,350.951753,2.157157e+07,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2020-08-03,536.000000,541.500000,528.000000,539.000000,1.807600e+07,6.0,20.798319,16408120.4,22048203.8
2020-08-04,546.000000,556.500000,540.000000,550.000000,2.018119e+07,19.0,20.830750,16285847.0,23188299.8
2020-08-05,557.500000,561.000000,551.000000,561.000000,1.588498e+07,6.5,20.738986,16015174.8,21675196.0


In [35]:
pd.set_option('display.max_rows', None)

df = data
n = 60
df['upper'],df['middle'],df['lower'] = ba.algo.ind.BBands(df)
df['Ret']= df['Close'].pct_change()
df['R_Std'] = df['Ret'].rolling(window=n).std()

df['Signal']=0
df.loc[(df['upper'].shift(2)>df['Close'].shift(2))&(df['upper'].shift(1)<df['Close'].shift(1)) \
  & (df['R_Std'].shift(1)>df['Ret'].shift(1)),'Signal']=1
df.loc[(df['lower'].shift(2)<df['Close'].shift(2))&(df['lower'].shift(1)>df['Close'].shift(1)) \
  & (-df['R_Std'].shift(1)<df['Ret'].shift(1)),'Signal']=-1

In [36]:
df['Str_ret']=df['Signal']*df['Ret']
df

,Open,High,Low,Close,Vol,pr_chg,std_100day,5d_avg_vol,past 5d_avg_vol,upper,middle,lower,Signal,Ret,R_Std,Str_ret
Date,,,,,,,,,,,,,,,,
2018-07-31,358.905334,359.700692,351.349432,353.138988,3.719448e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
2018-08-01,358.706494,359.104173,350.951753,352.940148,2.585878e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,-0.000563,NaN,-0.000000
2018-08-02,350.752913,352.343629,338.424863,342.998172,4.101500e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,-0.028169,NaN,-0.000000
2018-08-03,346.179604,352.542469,344.588888,347.770320,3.044742e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.013913,NaN,0.000000
2018-08-06,355.723901,355.723901,347.969160,350.951753,2.157157e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.009148,NaN,0.000000
2018-08-07,352.940148,356.320420,346.577283,354.928543,1.823712e+07,NaN,NaN,3.121745e+07,NaN,NaN,NaN,NaN,0,0.011331,NaN,0.000000
2018-08-08,364.472840,364.472840,358.507655,361.887926,2.559825e+07,1.789556,NaN,2.742598e+07,NaN,NaN,NaN,NaN,0,0.019608,NaN,0.000000
2018-08-09,364.870519,373.221779,362.882124,370.438026,2.678191e+07,8.947778,NaN,2.737387e+07,NaN,NaN,NaN,NaN,0,0.023626,NaN,0.000000
2018-08-10,373.221779,373.420619,362.484445,367.853112,1.963946e+07,27.439854,NaN,2.452725e+07,NaN,NaN,NaN,NaN,0,-0.006978,NaN,-0.000000
